# Homework #3: PCA/Hyperparameter/CV
Data source: http://archive.ics.uci.edu/ml/datasets/Polish+companies+bankruptcy+data

In [27]:
from scipy.io import arff
import pandas as pd
import numpy as np

In [28]:
data = arff.loadarff('D:\github代码\PHBS_MLF_2021-1\Data_Set/4year.arff')
df = pd.DataFrame(data[0])

In [29]:
df['bankruptcy'] = (df['class']==b'1')
df.drop(columns=['class'], inplace=True)
df.columns = ['X{0:02d}'.format(k) for k in range(1,65)] + ['bankruptcy']

In [30]:
df.head()

,X01,X02,X03,X04,X05,X06,X07,X08,X09,X10,...,X56,X57,X58,X59,X60,X61,X62,X63,X64,bankruptcy
0,0.159290,0.46240,0.07773,1.1683,-44.853,0.467020,0.189480,0.82895,1.1223,0.38330,...,0.108990,0.41557,0.89101,0.001422,7.7928,4.9914,119.810,3.0465,3.0560,False
1,-0.127430,0.46243,0.26917,1.7517,7.597,0.000925,-0.127430,1.16250,1.2944,0.53757,...,-0.089372,-0.23704,1.06250,0.150410,5.4327,3.4629,100.970,3.6150,3.4725,False
2,0.070488,0.23570,0.52781,3.2393,125.680,0.163670,0.086895,2.87180,1.0574,0.67689,...,0.054286,0.10413,0.94571,0.000000,7.1070,3.3808,76.076,4.7978,4.7818,False
3,0.136760,0.40538,0.31543,1.8705,19.115,0.504970,0.136760,1.45390,1.1144,0.58938,...,0.102630,0.23203,0.89737,0.073024,6.1384,4.2241,88.299,4.1337,4.6484,False
4,-0.110080,0.69793,0.18878,1.2713,-15.344,0.000000,-0.110080,0.43282,1.7350,0.30207,...,0.439880,-0.36440,0.57153,0.000000,18.8010,2.7925,146.390,2.4934,15.0360,False


In [31]:
sum(df.bankruptcy == True)

515

In [32]:
df.fillna(df.mean(), inplace=True)
df.isna().sum()
X_imp = df.values

In [33]:
from sklearn.model_selection import train_test_split

X, y = X_imp[:, :-1], X_imp[:, -1]

X_train, X_test, y_train, y_test =\
    train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In [34]:
import sklearn.preprocessing as skpre

stdsc = skpre.StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
print(X_train_std.shape)
X_test_std = stdsc.transform(X_test)
print(X_test_std.shape)

(6854, 64)
(2938, 64)


## Find the 2 most important features
using Logistic Regression with L1 penalty

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
# and put more...

In [23]:
lr = LogisticRegression(penalty = 'l1', C = 0.01, solver = 'liblinear', ) # complete
y_train = y_train.astype(int)
lr.fit(X_train_std, y_train)
print('Training accuracy:', lr.score(X_train_std, y_train))

Training accuracy: 0.9474759264662971


In [24]:
lr.coef_[lr.coef_!=0].shape # check the number of the features with non-zero weights

(2,)

In [25]:
# redefine X_train_std and X_test_std
X_train_std = X_train_std[:, lr.coef_[0]!=0]
X_test_std = X_test_std[:, lr.coef_[0]!=0]

## PCA using pipeline

In [67]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
pipe_lr_logistic = make_pipeline(StandardScaler(),
                     PCA(n_components=3),
                     LogisticRegression(random_state=1, solver='lbfgs'))
pipe_lr_svm = make_pipeline(StandardScaler(),
                     PCA(n_components=3),
                     SVC(kernel = 'linear', C = 1.0, random_state=1))
pipe_lr_tree = make_pipeline(StandardScaler(),
                     PCA(n_components=3),
                     DecisionTreeClassifier(criterion='gini', max_depth=4, random_state=1))
y_train = y_train.astype(int)
y_test = y_test.astype(int)
pipe_lr_logistic.fit(X_train,y_train)
pipe_lr_svm.fit(X_train,y_train)
pipe_lr_tree.fit(X_train,y_train)
y_pred_logistic = pipe_lr.predict(X_test)
y_pred_svm = pipe_lr.predict(X_test)
y_pred_tree = pipe_lr.predict(X_test)
print('Test Accuracy: %.3f,%.3f,%.3f' %(pipe_lr_logistic.score(X_test,y_test),
                                        pipe_lr_svm.score(X_test,y_test),
                                        pipe_lr_tree.score(X_test,y_test)))

Test Accuracy: 0.946,0.947,0.947


## 5 fold Cross-Validation

In [69]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

# normal form
kfold = StratifiedKFold(n_splits=5).split(X_train,y_train)
scores = []
for k, (train,test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr_logistic.score(X_train[test],y_train[test])
    scores.append(score)
    print('Fold: %2d, Class dis.: %s, Acc: %.3f' % (k+1,np.bincount(y_train[train]),score))
    
# sklearn form
scores_logistic = cross_val_score(estimator = pipe_lr_logistic,
                        X=X_train,
                        y=y_train,
                        cv=5,
                        n_jobs=1)
scores_svm = cross_val_score(estimator = pipe_lr_svm,
                        X=X_train,
                        y=y_train,
                        cv=5,
                        n_jobs=1)
scores_tree = cross_val_score(estimator = pipe_lr_tree,
                        X=X_train,
                        y=y_train,
                        cv=5,
                        n_jobs=1)
print('CV accuracy scores:%s,%s,%s'%(scores_logistic,scores_svm,scores_tree))
print('CV accuracy: %.3f +/- %.3f, %.3f +/- %.3f  ,   %.3f +/- %.3f   '%(np.mean(scores_logistc),
                                   np.std(scores_logistc),np.mean(scores_svm),
                                   np.std(scores_svm),np.mean(scores_tree),
                                   np.std(scores_tree)))



Fold:  1, Class dis.: [5195  288], Acc: 0.947
Fold:  2, Class dis.: [5195  288], Acc: 0.947
Fold:  3, Class dis.: [5195  288], Acc: 0.946
Fold:  4, Class dis.: [5195  288], Acc: 0.948
Fold:  5, Class dis.: [5196  288], Acc: 0.946
CV accuracy scores:[0.94675419 0.94748359 0.9460248  0.94748359 0.94671533],[0.94748359 0.94748359 0.94748359 0.94748359 0.94744526],[0.94748359 0.94748359 0.94748359 0.94675419 0.94671533]
CV accuracy: 0.947 +/- 0.001, 0.947 +/- 0.000  ,   0.947 +/- 0.000   


## Grid search

In [70]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# svm
param_range = [0.0001, 0.001, 0.01, 0.1,
              1.0, 10.0, 100.0, 1000.0]
parameters = {'svc__kernel':('linear', 'rbf'), 'svc__C':param_range}
gs_svm = GridSearchCV(estimator=pipe_lr_svm,
                     param_grid=parameters,
                     scoring='accuracy',
                     cv=5,
                     refit=True,
                     n_jobs=1)
pipe_lr_svm.get_params().keys()
gs_svm = gs_svm.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params)

In [ ]:
# tree 
param_range = [3,4,5]
pipe_lr_tree.get_params().keys()
parameters = {'':('entropy','gini'),'':param_range}
gs_tree = GridSearchCV(estimator=pipe_lr_tree,
                     param_grid=parameters,
                     scoring='accuracy',
                     cv=5,
                     refit=True,
                     n_jobs=1)
gs_tree = gs_tree.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params)

## Visualize the classification

In [ ]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

def plot_decision_regions(X, y, classifier, test_idx=None, resolution=0.02):

    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], 
                    y=X[y == cl, 1],
                    alpha=0.8, 
                    c=colors[idx],
                    marker=markers[idx], 
                    label=cl, 
                    edgecolor='black')

    # highlight test samples
    if test_idx:
        # plot all samples
        X_test, y_test = X[test_idx, :], y[test_idx]

        plt.scatter(X_test[:, 0],
                    X_test[:, 1],
                    c='',
                    edgecolor='black',
                    alpha=1.0,
                    linewidth=1,
                    marker='o',
                    s=100, 
                    label='test set')

In [ ]:
X_combined_std = np.vstack((X_train_std, X_test_std))
y_combined = np.hstack((y_train, y_test))

plot_decision_regions(X=X_combined_std, y=y_combined,
                      classifier=lr, test_idx=range(0, 50))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')

plt.tight_layout()
#plt.savefig('images/03_01.png', dpi=300)
plt.show()